<a href="https://colab.research.google.com/github/qkrtnwjd4212/Time_Series_Data_Project/blob/main/TA10_GA_and_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.25.2 --force-reinstall

In [ ]:
!pip install pandas_ta yfinance --upgrade

In [ ]:
!pip install pygad backtesting

In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import pygad
from backtesting import Backtest, Strategy, set_bokeh_output

In [ ]:
#gdrive 마운트
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd "/content/gdrive/MyDrive/Colab Notebooks/stock"

In [ ]:
ls

 All_TA_signals.csv          'TA signal data.ipynb'
 backtest_results.csv         test_results_2024.csv
 GA_result_all_tickers.csv    test_results_2024_sim_equity_final_kor.csv
 TA_data_30.csv               tictocstock_2021_2024.csv
 TA_Feature_Selection.ipynb   tictocstock.csv
 TA_signal_counts_2.csv      '초기 feature 코드 '
 TA_signal_counts.csv


In [ ]:
# # 1) 다운로드할 종목 리스트와 기간 설정
# tickers    = ["HD", "MCD", "LOW", "TJX", "SBUX", "NKE", "MAR", "CMG", "ORLY", "TGT"]
# start_date = "2021-01-01"
# end_date   = "2024-12-31"   # 2024년 말까지

# # 2) 각 티커별로 데이터 다운로드 후 리스트에 담기
# frames = []
# for ticker in tickers:
#     df = yf.download(
#         ticker,
#         start=start_date,
#         end=end_date,
#         auto_adjust=False  # Adj Close 컬럼을 별도로 받기 위함
#     )
#     # 멀티인덱스 컬럼 제거
#     if isinstance(df.columns, pd.MultiIndex):
#         df.columns = df.columns.droplevel(level=1)

#     # Date 컬럼으로 올리고 Ticker 추가
#     df = df.reset_index()
#     df["Ticker"] = ticker

#     # 필요한 컬럼만 추출, Adj Close를 Close로 이름 변경
#     df = df[["Date", "Ticker", "Open", "High", "Low", "Adj Close", "Volume"]]
#     df = df.rename(columns={"Adj Close": "Close"})

#     frames.append(df)

# # 3) 모두 합쳐서 CSV로 저장
# all_data = pd.concat(frames, ignore_index=True)
# all_data.to_csv("tictocstock_2021_2024.csv", index=False)

# print("tictocstock_2021_2024.csv 생성 완료")


In [ ]:
# 1) 설정
tickers    = ["HD", "MCD", "LOW", "TJX", "SBUX", "NKE", "MAR", "CMG", "ORLY", "TGT"]
start_date = "2021-01-01"
end_date   = "2024-01-01"

In [ ]:
INITIAL_CASH   = 100_000
COMMISSION_RATE = 0.01

def simulate_equity(price, signal, cash=INITIAL_CASH, commission=COMMISSION_RATE):
    """
    price : 1D array of 거래 가격 (Adj Close)
    signal: 1D array of 시그널 (-1=buy, 1=sell, 0=hold)
    returns: profit (총 수익률), mdd (최대 낙폭)
    """
    n = len(price)
    equity_curve = np.zeros(n)
    equity = cash
    shares = 0.0
    in_pos = False

    # 시그널이 없거나 가격 데이터가 부족한 경우 처리
    if n == 0 or len(signal) == 0 or len(price) != len(signal):
        return 0.0, 1.0 # 최소 수익, 최대 MDD 반환 (패널티)

    for i in range(n):
        sig = signal[i]
        p   = price[i]

        # 매수
        if sig == -1 and not in_pos and equity > 0: # 자산이 있어야 매수 가능
            entry_price = p * (1 + commission)
            if entry_price > 0: # 0으로 나누기 방지
                shares = equity / entry_price
                in_pos = True
            else: # 매수 불가시 현재 상태 유지
                equity_curve[i] = equity
                continue

        # 매도
        elif sig == 1 and in_pos:
            exit_price = p * (1 - commission)
            equity = shares * exit_price
            shares = 0.0
            in_pos = False

        # 일간 지분가치 기록
        equity_curve[i] = (shares * p) if in_pos else equity

    # 마지막에 아직 포지션이 남아있으면 청산
    if in_pos and len(price) > 0 : # price가 비어있지 않은지 확인
        exit_price = price[-1] * (1 - commission)
        equity = shares * exit_price
        equity_curve[-1] = equity

    if cash == 0: # 초기 자본이 0인 경우 수익률 계산 불가
        profit = 0.0
    else:
        profit = equity / cash - 1.0

    # MDD 계산 시 equity_curve가 모두 0이거나 NaN인 경우 처리
    if np.all(equity_curve == 0) or np.all(np.isnan(equity_curve)):
        mdd = 1.0 # 최대 MDD
    else:
        peak = np.maximum.accumulate(equity_curve)
        # peak가 0인 경우를 방지하기 위해 작은 값(epsilon)을 더하거나, 0인 경우 dd를 0으로 설정
        epsilon = 1e-9
        peak_safe = np.where(peak == 0, epsilon, peak)
        dd = (peak_safe - equity_curve) / peak_safe
        # dd 계산 결과 NaN이 있을 경우 0으로 처리 (예: peak와 equity_curve가 모두 0인 경우)
        dd = np.nan_to_num(dd, nan=0.0)
        mdd = np.max(dd) if len(dd) > 0 else 1.0

    # profit이나 mdd가 비정상적인 값(NaN, inf)을 가질 경우 패널티
    if not np.isfinite(profit):
        profit = -1.0 # 매우 낮은 수익률로 간주
    if not np.isfinite(mdd) or mdd < 0 or mdd > 1: # MDD는 0과 1 사이 값
        mdd = 1.0 # 최대 MDD

    return profit, mdd

In [ ]:
def evaluate_strategy(indicator_name, params, price_df):
    df = price_df.copy()
    # ================================ 지표별 시그널 계산 ================================
    if indicator_name == "RSI":
        length = int(round(params[0]))
        if length <= 1: length = 2 # RSI 길이는 1보다 커야 함
        df["ind"] = ta.rsi(df["Close"], length=length)
        df["signal"] = 0
        df.loc[df["ind"] < 30, "signal"] = -1
        df.loc[df["ind"] > 70, "signal"] = 1
    else:
        if indicator_name == "SMA":
            short_len, long_len = map(lambda x: int(round(x)), params)
            if short_len <= 0: short_len = 1
            if long_len <= short_len: long_len = short_len + 1 # long_len은 short_len보다 커야 함
            ma_short = ta.sma(df["Close"], length=short_len)
            ma_long = ta.sma(df["Close"], length=long_len)
            df["ind"] = ma_short - ma_long

        elif indicator_name == "EMA":
            short_len, long_len = map(lambda x: int(round(x)), params)
            if short_len <= 0: short_len = 1
            if long_len <= short_len: long_len = short_len + 1
            ma_short = ta.ema(df["Close"], length=short_len)
            ma_long = ta.ema(df["Close"], length=long_len)
            df["ind"] = ma_short - ma_long

        elif indicator_name == "MACD":
            fast, slow = map(lambda x: int(round(x)), params)
            if fast <= 0: fast = 1
            if slow <= fast: slow = fast + 1
            # signal period는 기본값 9 사용 명시
            macd_df = ta.macd(df["Close"], fast=fast, slow=slow, signal=9)
            if macd_df is None or macd_df.shape[1] < 2: # MACD 계산 실패시
                df["ind"] = 0
            else:
                macd_line = macd_df.iloc[:, 0]  # MACD Line
                macd_signal_line = macd_df.iloc[:, 1] # MACD Signal Line
                df["ind"] = macd_line - macd_signal_line

        elif indicator_name == "STOCH":
            k, d = map(lambda x: int(round(x)), params)
            if k <= 0: k = 1
            if d <= 0: d = 1
            # STOCH의 경우 high, low, close가 필요
            st = ta.stoch(df["High"], df["Low"], df["Close"], k=k, d=d, smooth_k=3) # smooth_k는 일반적 값
            if st is None or st.shape[1] < 2:
                df["ind"] = 0
            else:
                k_s = st.iloc[:, 0] # STOCHk
                d_s = st.iloc[:, 1] # STOCHd
                df["ind"] = k_s - d_s

        elif indicator_name == "ROC":
            length = int(round(params[0]))
            if length <= 0: length = 1
            df["ind"] = ta.roc(df["Close"], length=length)

        elif indicator_name == "BBANDS":
          length = int(round(params[0]))
          std = float(params[1]) # GA는 length와 std 두 파라미터를 최적화
          if length <= 1: length = 2
          if std <= 0: std = 0.1 # 표준편차는 0보다 커야 함

          # 볼린저 밴드 계산
          bb_df = ta.bbands(df["Close"], length=length, std=std)

          if bb_df is None or bb_df.shape[1] < 3:
              df["ind"] = 0    # 지표 계산 실패 시 기본값
              df["signal"] = 0 # 시그널 없음
          else:
              lower_band = bb_df.iloc[:, 0]
              upper_band = bb_df.iloc[:, 2]
              band_width = upper_band - lower_band
              # band_width가 0이 되는 경우를 방지
              df["ind"] = (df["Close"] - lower_band) / band_width.replace(0, np.nan) # 0으로 나누기 방지

              # 시그널 생성
              df["signal"] = 0  # 기본값: 홀드

              # 매수 시그널 (-1): 종가가 하단 밴드 아래로 내려갔을 때 (과매도)
              df.loc[df["Close"] < lower_band, "signal"] = -1

              # 매도 시그널 (1): 종가가 상단 밴드 위로 올라갔을 때 (과열)
              df.loc[df["Close"] > upper_band, "signal"] = 1

        elif indicator_name == "ATR":
          # ATR 자체를 계산하기 위한 기간 (GA 최적화 파라미터)
          atr_period = int(round(params[0]))
          if atr_period <= 0: atr_period = 1 # 최소 기간 1로 보정
          center_line_ma_period = 20  # 중심선 계산을 위한 이동평균 기간
          atr_channel_multiplier = 2.0 # ATR 값에 곱해줄 배수 (채널 폭 결정)

          # 필요한 지표 계산
          df["atr_value"] = ta.atr(df["High"], df["Low"], df["Close"], length=atr_period)
          df["center_line"] = ta.sma(df["Close"], length=center_line_ma_period) # 중심선 (SMA 사용) <- 여기에 들어가는 len도 최적화되어야

          df["upper_channel"] = df["center_line"] + (atr_channel_multiplier * df["atr_value"])
          df["lower_channel"] = df["center_line"] - (atr_channel_multiplier * df["atr_value"])
          df["ind"] = df["atr_value"]

          # 시그널 생성
          df["signal"] = 0  # 기본값: 홀드 (거래 없음)

          # 이전 날짜 값 계산 (돌파 확인용)
          prev_close = df["Close"].shift(1)
          prev_upper_channel = df["upper_channel"].shift(1)
          prev_lower_channel = df["lower_channel"].shift(1)

          # 매수 시그널 (-1): 종가가 상단 채널을 상향 돌파
          # 조건: (현재 종가 > 현재 상단 채널) AND (이전 종가 <= 이전 상단 채널)
          buy_condition = (df["Close"] > df["upper_channel"]) & (prev_close <= prev_upper_channel)
          df.loc[buy_condition, "signal"] = -1

          # 매도 시그널 (1): 종가가 하단 채널을 하향 돌파
          # 조건: (현재 종가 < 현재 하단 채널) AND (이전 종가 >= 이전 하단 채널)
          sell_condition = (df["Close"] < df["lower_channel"]) & (prev_close >= prev_lower_channel)
          df.loc[sell_condition, "signal"] = 1

        elif indicator_name == "OBV":
            smooth = int(round(params[0]))
            if smooth <= 0: smooth = 1
            obv_val = ta.obv(df["Close"], df["Volume"])
            df["ind"] = obv_val - ta.sma(obv_val, length=smooth) # OBV와 OBV SMA의 차이

        elif indicator_name == "CMF":
            length = int(round(params[0]))
            if length <= 0: length = 1
            df["ind"] = ta.cmf(df["High"], df["Low"], df["Close"], df["Volume"], length=length)
        else:
            raise ValueError(f"Unknown indicator: {indicator_name}")


        # --- 최종 시그널 정리 ---
        if indicator_name not in ["RSI", "ATR", "BBANDS"]: # RSI와 ATR이 아닌 경우
            if "ind" in df.columns:
                df["signal"] = np.where(df["ind"] > 0, -1,  # ind > 0 이면 매수
                                        np.where(df["ind"] < 0, 1, 0)) # ind < 0 이면 매도
            else: # "ind" 컬럼이 없는 예외적인 경우 (모든 지표는 ind를 설정해야 함)
                df["signal"] = 0 # 안전하게 홀드

    df.fillna(0, inplace=True) # 최종적으로 NaN 값 처리

    # 시그널 생성 과정에서 발생할 수 있는 NaN을 0(홀드)으로 처리
    if "signal" in df.columns:
        df["signal"] = df["signal"].fillna(0)
    else: # "signal" 컬럼이 아예 없는 경우 (예: 로직 오류)
        df["signal"] = 0 # 안전하게 홀드


   # ================================ 수익률 & MDD 계산 및 fitness 평가 ================================
    price_arr  = df["Close"].values
    signal_arr = df["signal"].values

    # simulate_equity 함수를 호출하여 실제 수익률(profit_sim)과 MDD(mdd_sim) 계산
    profit_sim, mdd_sim = simulate_equity(price_arr, signal_arr)

    # MDD가 0이거나 매우 작은 경우 0으로 나누기 오류 방지 및 극단적인 fitness 값 방지
    epsilon = 1e-9  # 매우 작은 양수 값
    safe_mdd = max(mdd_sim, epsilon)

    if profit_sim > 0: # 수익률이 양수일때만 fitness 공식 적용
        fitness_value = (0.8 * profit_sim) + (0.2 * (1 / safe_mdd))
    else:
        # 수익률이 0이거나 음수인 경우 1/MDD 항의 긍정적 효과가 전체 평가를 왜곡하지 않도록 처리
        fitness_value = profit_sim - (0.2 * mdd_sim) # 손실일 때는 MDD가 클수록 더 낮은 fitness

    # 최종 fitness 값의 유효성 검사 (NaN 또는 무한대 방지)
    if not np.isfinite(profit_sim) or not np.isfinite(mdd_sim):
        fitness_value = -float('inf')
    elif not np.isfinite(fitness_value):
        fitness_value = -float('inf')

    return fitness_value # 높을 수록 좋은 값

In [ ]:
def run_ga(indicator_name, price_df):

    if indicator_name in ["SMA", "EMA"]:
        unified_ma_range = {"low": 5, "high": 200}  # 예: 5일 ~ 200일 범위
        gene_space = [
            unified_ma_range,  # 첫 번째 MA 길이 (p1)
            unified_ma_range   # 두 번째 MA 길이 (p2)
        ]
        num_genes = 2
        gene_type = [int, int]
    elif indicator_name == "MACD":
        unified_ema_range_for_macd = {"low": 5, "high": 200} # 예: 5일 ~ 200일 범위
        gene_space = [
            unified_ema_range_for_macd, # 첫 번째 EMA 기간 (p1)
            unified_ema_range_for_macd  # 두 번째 EMA 기간 (p2)
        ]
        num_genes = 2
        gene_type = [int, int]
    elif indicator_name == "STOCH":
        gene_space = [
            {"low": 5,  "high": 30},  # k 기간
            {"low": 3,  "high": 10}   # d 기간
        ]
        num_genes = 2
        gene_type = [int, int]
    elif indicator_name == "BBANDS":
        gene_space = [
            {"low": 10, "high": 50},  # 중심 밴드 기간 (int)
            {"low": 1.0,"high": 3.0}  # 표준편차 배수 (float)
        ]
        num_genes = 2
        gene_type = [int, float]
    else: # 단일 파라미터 (RSI, ROC, ATR, OBV, CMF)
        gene_space = [{"low": 5,  "high": 200}]
        num_genes = 1
        gene_type = int
        if indicator_name == "ATR":
            # ATR 채널 전략의 ATR period를 최적화합니다.
            # (중심선 MA 기간 및 채널 배수는 evaluate_strategy 내에 고정값으로 설정되어 있다고 가정)
            pass # 특별한 gene_space 변경 없음

    def fitness_fun_wrapper(ga_instance, solution, solution_idx):
        return evaluate_strategy(indicator_name, solution, price_df)

    ga = pygad.GA(
        gene_space=gene_space,
        fitness_func=fitness_fun_wrapper, # 수정된 wrapper 사용
        sol_per_pop=20,
        num_generations=30, # 50으로 키워보기
        num_genes=num_genes,
        gene_type=gene_type, # gene_type 명시
        num_parents_mating=10,
        parent_selection_type="sss",
        crossover_type="single_point",
        mutation_type="random",
        mutation_percent_genes=10,
        # 정수형 유전자이고 범위가 작을 경우 mutation_by_replacement=True 고려
        # keep_elitism=1 # 최고의 해는 다음 세대로 보존
    )
    ga.run()

    # best_solution()은 (solution, fitness, index) 튜플을 반환
    raw_solution, fitness, _ = ga.best_solution()

    solution = []
    for i,gene_val in enumerate(raw_solution):
        if isinstance(gene_type, list): # gene_type이 리스트인 경우
            if gene_type[i] == int:
                solution.append(int(round(gene_val)))
            else:
                solution.append(float(gene_val))
        else: # gene_type이 단일 타입인 경우
            if gene_type == int:
                solution.append(int(round(gene_val)))
            else: # float 등 다른 타입
                solution.append(float(gene_val))

    return solution, fitness

In [ ]:

results = []
tickers    = ["HD", "MCD", "LOW", "TJX", "SBUX", "NKE", "MAR", "CMG", "ORLY", "TGT"]

df_all = pd.read_csv("tictocstock_2021_2024.csv", parse_dates=["Date"])
df_all.sort_values(["Ticker", "Date"], inplace=True)

# 전체 종목에 대해 GA 실행
for ticker in tickers:
    # 해당 종목 데이터 전처리
    df_price = df_all[df_all["Ticker"] == ticker].copy()
    df_price.set_index("Date", inplace=True)
    df_price = df_price.loc["2021-01-01":"2023-12-31", ["Open", "High", "Low", "Close", "Volume"]]

    # 각 지표별 GA 최적화
    for ind in ["SMA", "EMA", "MACD", "RSI", "STOCH", "ROC", "BBANDS", "ATR", "OBV", "CMF"]:
        best_params, best_fit = run_ga(ind, df_price)
        results.append({
            "Ticker":      ticker,
            "Indicator":   ind,
            "Best_Params": best_params,
            "Best_Fitness": best_fit
        })

# 결과 DataFrame 생성 및 저장
results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv("GA_result_all_tickers.csv", index=False)

In [ ]:
# 터닝 포인트 감지
def find_turning_points(prices, time_intervals, T, P):
    """
    변화율이 P보다 크면 터닝 포인트로 인식하는 방식.
    """

    turning_points = [0]  # 시작점은 항상 포함
    i = 0

    while i < len(prices) - 1:
        j = i + 1
        if j >= len(prices):
            break

        # 시간 간격이 충분하지 않으면 다음으로
        if time_intervals[j] < T:
            i += 1
            continue

        # 변화율 계산
        avg_price = (prices[i] + prices[j]) / 2
        percentage_change = abs(prices[j] - prices[i]) / avg_price if avg_price != 0 else 0

        # 변화율이 P 이상이면 터닝 포인트 인정
        if percentage_change >= P:
            turning_points.append(j)
            i = j  # 새로운 기준점으로 이동
        else:
            i += 1

    # 마지막 포인트는 항상 포함
    if turning_points[-1] != len(prices) - 1:
        turning_points.append(len(prices) - 1)

    return turning_points

def calculate_percentage_changes(prices):
    """주가 데이터로부터 가격 변화율을 계산하는 함수."""
    changes = []
    for i in range(len(prices) - 1):
        avg_price = (prices[i] + prices[i + 1]) / 2
        percentage_change = abs(prices[i + 1] - prices[i]) / avg_price if avg_price != 0 else 0
        changes.append(percentage_change)
    return changes

In [ ]:
# GA 파라미터 뽑은거로 24년 데이터 백테스트 진행하는 코드
# 수정 필요 (백테스트 라이브러리 사용안했음, 시그널 데이터 따로 csv파일 만들어서 진행하는거로 고치기)

import ast

# GA 최적화 시 사용된 상수
INITIAL_CASH    = 100_000
COMMISSION_RATE = 0.01
PROFIT_WEIGHT   = 0.8 # evaluate_strategy의 fitness 계산 가중치와 일치해야 함
MDD_WEIGHT      = 0.2 # evaluate_strategy의 fitness 계산 가중치와 일치해야 함

def simulate_equity(price, signal, cash=INITIAL_CASH, commission=COMMISSION_RATE):
    n = len(price)
    equity_curve = np.zeros(n)
    equity = cash
    shares = 0.0
    in_pos = False
    if n == 0 or len(signal) == 0 or len(price) != len(signal):
        return 0.0, 1.0 # 최소 수익, 최대 MDD (패널티)

    for i in range(n):
        sig = signal[i]
        p   = price[i]
        # 매수
        if sig == -1 and not in_pos and equity > 0: # 자산이 있어야 매수 가능
            entry_price = p * (1 + commission)
            if entry_price > 0: # 0으로 나누기 방지
                shares = equity / entry_price
                in_pos = True
            else: # 매수 불가시 현재 상태 유지
                equity_curve[i] = equity
                continue
        # 매도
        elif sig == 1 and in_pos:
            exit_price = p * (1 - commission)
            equity = shares * exit_price
            shares = 0.0
            in_pos = False

        equity_curve[i] = (shares * p) if in_pos else equity

    if in_pos and len(price) > 0 : # price 배열이 비어있지 않은지 확인
        exit_price = price[-1] * (1 - commission)
        equity = shares * exit_price
        equity_curve[-1] = equity
    if cash == 0: # 초기 자본이 0인 경우 수익률 계산 불가
        profit = 0.0
    else:
        profit = equity / cash - 1.0
    # MDD 계산 시 equity_curve가 모두 0이거나 NaN인 경우 처리
    if np.all(equity_curve == 0) or np.all(np.isnan(equity_curve)):
        mdd = 1.0 # 최대 MDD
    else:
        peak = np.maximum.accumulate(equity_curve)
        # peak가 0인 경우를 방지하기 위해 작은 값(epsilon)을 더함
        epsilon = 1e-9
        peak_safe = np.where(peak == 0, epsilon, peak)
        dd = (peak_safe - equity_curve) / peak_safe
        # dd 계산 결과 NaN이 있을 경우 처리 (예: peak와 equity_curve가 모두 0인 경우)
        dd = np.nan_to_num(dd, nan=0.0)
        mdd = np.max(dd) if len(dd) > 0 else 1.0
    # 유효하지 않은 profit 또는 mdd 값에 패널티 부여
    if not np.isfinite(profit):
        profit = -1.0 # 매우 낮은 수익률로 간주
    if not np.isfinite(mdd) or mdd < 0 or mdd > 1: # MDD는 0과 1 사이 값이어야 함
        mdd = 1.0 # 최대 MDD
    return profit, mdd

# 2) 데이터 및 GA 결과 로드
df_all_data     = pd.read_csv("tictocstock_2021_2024.csv", parse_dates=["Date"])
# GA 결과 파일명이 정확한지 확인
ga_results_df = pd.read_csv("GA_result_all_tickers.csv")
ga_results_df["Best_Params"] = ga_results_df["Best_Params"].apply(ast.literal_eval)

# 3) 백테스트 결과 저장을 위한 리스트
backtest_results_list = []

# 4) 테스트 대상 지표 리스트
indicators_to_test_list = ["SMA", "EMA", "MACD", "RSI", "STOCH", "ROC", "BBANDS", "ATR", "CMF", "OBV"]

# 5) 전체 티커 및 선택된 지표에 대해 백테스팅 수행
for ticker_symbol in df_all_data["Ticker"].unique():
    print(f"백테스팅 시작: {ticker_symbol}")
    df_ticker_full_period = df_all_data[df_all_data["Ticker"] == ticker_symbol].copy()
    df_ticker_full_period.set_index("Date", inplace=True)

    # 테스트 기간 데이터 (2024년)
    df_test_period = df_ticker_full_period.loc[
        (df_ticker_full_period.index >= "2024-01-01") & (df_ticker_full_period.index <= "2024-12-31"),
        ["Open", "High", "Low", "Close", "Volume"] # 필요한 컬럼만 선택
    ].copy()

    for indicator_name_loop in indicators_to_test_list:
        # 현재 티커와 지표에 대한 GA 최적 파라미터 가져오기
        ga_params_row = ga_results_df.loc[
            (ga_results_df["Ticker"] == ticker_symbol) &
            (ga_results_df["Indicator"] == indicator_name_loop)
        ]

        optimal_parameters = ga_params_row["Best_Params"].iloc[0]

        # df_for_evaluation은 evaluate_strategy의 price_df에 해당
        df_for_evaluation = df_test_period.copy()

        # --- evaluate_strategy 로직 (지표 계산 및 시그널 생성) ---
        if indicator_name_loop == "RSI":
            length = int(round(optimal_parameters[0]))
            if length <= 1: length = 2
            df_for_evaluation["ind"] = ta.rsi(df_for_evaluation["Close"], length=length)
            df_for_evaluation["signal"] = 0
            df_for_evaluation.loc[df_for_evaluation["ind"] < 30, "signal"] = -1 # 매수
            df_for_evaluation.loc[df_for_evaluation["ind"] > 70, "signal"] = 1  # 매도

        elif indicator_name_loop == "ATR":
            atr_period = int(round(optimal_parameters[0])) # GA는 ATR 기간만 최적화
            if atr_period <= 0: atr_period = 1

            # ATR 채널을 위한 추가 파라미터 (evaluate_strategy와 동일한 하드코딩 값 사용)
            center_line_ma_period = 20
            atr_channel_multiplier = 2.0

            df_for_evaluation["atr_value"] = ta.atr(df_for_evaluation["High"], df_for_evaluation["Low"], df_for_evaluation["Close"], length=atr_period)
            df_for_evaluation["center_line"] = ta.sma(df_for_evaluation["Close"], length=center_line_ma_period)
            df_for_evaluation["upper_channel"] = df_for_evaluation["center_line"] + (atr_channel_multiplier * df_for_evaluation["atr_value"])
            df_for_evaluation["lower_channel"] = df_for_evaluation["center_line"] - (atr_channel_multiplier * df_for_evaluation["atr_value"])

            # 'ind' 컬럼은 ATR 값 자체로 설정 (evaluate_strategy와 일관성)
            df_for_evaluation["ind"] = df_for_evaluation["atr_value"]

            df_for_evaluation["signal"] = 0 # 기본 홀드

            prev_close = df_for_evaluation["Close"].shift(1)
            prev_upper_channel = df_for_evaluation["upper_channel"].shift(1)
            prev_lower_channel = df_for_evaluation["lower_channel"].shift(1)

            buy_condition = (df_for_evaluation["Close"] > df_for_evaluation["upper_channel"]) & (prev_close <= prev_upper_channel)
            sell_condition = (df_for_evaluation["Close"] < df_for_evaluation["lower_channel"]) & (prev_close >= prev_lower_channel)

            df_for_evaluation.loc[buy_condition, "signal"] = -1
            df_for_evaluation.loc[sell_condition, "signal"] = 1

        elif indicator_name_loop == "BBANDS":
            length = int(round(optimal_parameters[0]))
            std_dev = float(optimal_parameters[1])
            if length <= 1: length = 2
            if std_dev <= 0: std_dev = 0.1

            bbands_calculated = ta.bbands(df_for_evaluation["Close"], length=length, std=std_dev)
            if bbands_calculated is None or bbands_calculated.shape[1] < 3:
                df_for_evaluation["ind"] = 0
                df_for_evaluation["signal"] = 0 # 밴드 계산 불가 시 시그널 없음
            else:
                lower_band = bbands_calculated.iloc[:, 0] # BBL (하단 밴드)
                upper_band = bbands_calculated.iloc[:, 2] # BBU (상단 밴드)

                # 'ind' 컬럼은 참고용으로 %B 값 등으로 설정 가능 (GA가 직접 사용하지 않음)
                band_width = upper_band - lower_band
                df_for_evaluation["ind"] = (df_for_evaluation["Close"] - lower_band) / band_width.replace(0, np.nan)
                df_for_evaluation["ind"] = df_for_evaluation["ind"].fillna(0.5) # 계산 불가시 중간값으로 가정

                # 시그널 직접 생성
                df_for_evaluation["signal"] = 0  # 기본값: 홀드
                df_for_evaluation.loc[df_for_evaluation["Close"] < lower_band, "signal"] = -1 # 매수
                df_for_evaluation.loc[df_for_evaluation["Close"] > upper_band, "signal"] = 1 # 매도

        else: # SMA, EMA, MACD, STOCH, ROC, BBANDS, CMF
            if indicator_name_loop == "SMA":
                short_len, long_len = map(lambda x: int(round(x)), optimal_parameters)
                if short_len <= 0: short_len = 1
                if long_len <= short_len: long_len = short_len + 1
                ma_short = ta.sma(df_for_evaluation["Close"], length=short_len)
                ma_long = ta.sma(df_for_evaluation["Close"], length=long_len)
                df_for_evaluation["ind"] = ma_short - ma_long

            elif indicator_name_loop == "EMA":
                short_len, long_len = map(lambda x: int(round(x)), optimal_parameters)
                if short_len <= 0: short_len = 1
                if long_len <= short_len: long_len = short_len + 1
                ma_short = ta.ema(df_for_evaluation["Close"], length=short_len)
                ma_long = ta.ema(df_for_evaluation["Close"], length=long_len)
                df_for_evaluation["ind"] = ma_short - ma_long

            elif indicator_name_loop == "MACD":
                fast, slow = map(lambda x: int(round(x)), optimal_parameters)
                if fast <= 0: fast = 1
                if slow <= fast: slow = fast + 1
                macd_df_calculated = ta.macd(df_for_evaluation["Close"], fast=fast, slow=slow, signal=9)
                if macd_df_calculated is None or macd_df_calculated.shape[1] < 2:
                    df_for_evaluation["ind"] = 0
                else:
                    macd_line = macd_df_calculated.iloc[:, 0]
                    macd_signal_line = macd_df_calculated.iloc[:, 1] # 시그널 라인이 두 번째 컬럼이라고 가정
                    df_for_evaluation["ind"] = macd_line - macd_signal_line

            elif indicator_name_loop == "STOCH":
                k, d = map(lambda x: int(round(x)), optimal_parameters)
                if k <= 0: k = 1
                if d <= 0: d = 1
                stoch_calculated = ta.stoch(df_for_evaluation["High"], df_for_evaluation["Low"], df_for_evaluation["Close"], k=k, d=d, smooth_k=3)
                if stoch_calculated is None or stoch_calculated.shape[1] < 2:
                    df_for_evaluation["ind"] = 0
                else:
                    k_s = stoch_calculated.iloc[:, 0] # %K
                    d_s = stoch_calculated.iloc[:, 1] # %D
                    df_for_evaluation["ind"] = k_s - d_s

            elif indicator_name_loop == "ROC":
                length = int(round(optimal_parameters[0]))
                if length <= 0: length = 1
                df_for_evaluation["ind"] = ta.roc(df_for_evaluation["Close"], length=length)

            elif indicator_name_loop == "CMF":
                length = int(round(optimal_parameters[0]))
                if length <= 0: length = 1
                df_for_evaluation["ind"] = ta.cmf(df_for_evaluation["High"], df_for_evaluation["Low"], df_for_evaluation["Close"], df_for_evaluation["Volume"], length=length)

            elif indicator_name_loop == "OBV":
                smooth = int(round(optimal_parameters[0]))
                if smooth <= 0: smooth = 1
                obv_val = ta.obv(df_for_evaluation["Close"], df_for_evaluation["Volume"])
                if obv_val is not None and isinstance(obv_val, pd.Series) and not obv_val.empty:
                    sma_obv = ta.sma(obv_val, length=smooth)
                    if sma_obv is not None: # Check if sma calculation is valid
                         df_for_evaluation["ind"] = obv_val - sma_obv
                    else:
                         df_for_evaluation["ind"] = 0 # SMA calculation failed
                else:
                    df_for_evaluation["ind"] = 0 # OBV calculation failed

            if "ind" in df_for_evaluation.columns:
                df_for_evaluation["ind"] = df_for_evaluation["ind"].fillna(0)
                df_for_evaluation["signal"] = np.where(df_for_evaluation["ind"] > 0, -1,
                                                     np.where(df_for_evaluation["ind"] < 0, 1, 0))
            else: # ind 컬럼이 어떤 이유로든 생성되지 않은 경우
                df_for_evaluation["signal"] = 0


            df_for_evaluation.fillna(0, inplace=True) # 'ind' 컬럼의 NaN 처리

            # 시그널 생성 (RSI가 아닌 지표용)
            df_for_evaluation["signal"] = np.where(df_for_evaluation["ind"] > 0, -1, # ind > 0 이면 매수
                                             np.where(df_for_evaluation["ind"] < 0, 1, 0)) # ind < 0 이면 매도
        # --- evaluate_strategy 로직 끝 ---

        df_for_evaluation.fillna(0, inplace=True) # 최종 NaN 처리 (주로 'signal' 컬럼)

        # 백테스팅을 위한 가격 및 시그널 배열 준비
        price_array = df_for_evaluation["Close"].values
        signal_array = df_for_evaluation["signal"].values

        if len(price_array) == 0: # 시그널 생성 후 데이터 없는 경우 (아마도 모두 NaN)
            print(f"시그널 생성 후 데이터 없음: {ticker_symbol} - {indicator_name_loop}. 다음으로 넘어갑니다.")
            continue

        # simulate_equity 함수를 사용하여 백테스트 수행
        profit_result, mdd_result = simulate_equity(price_array, signal_array, cash=INITIAL_CASH, commission=COMMISSION_RATE)

        # Fitness 계산 (GA 최적화 시와 동일한 방식)
        test_fitness_value = (PROFIT_WEIGHT * profit_result) - (MDD_WEIGHT * mdd_result)
        if not np.isfinite(test_fitness_value): # 유효하지 않은 fitness 값에 패널티 부여
            test_fitness_value = -float('inf')

        backtest_results_list.append({
            "Ticker": ticker_symbol,
            "Indicator": indicator_name_loop,
            "Params_GA": optimal_parameters, # GA로 찾은 파라미터 저장
            "Profit_2024": profit_result,
            "MDD_2024": mdd_result,
            "Fitness_2024": test_fitness_value
        })
        print(f"  완료: {indicator_name_loop} | 수익률: {profit_result:.4f}, MDD: {mdd_result:.4f}, Fitness: {test_fitness_value:.4f}")

# 6) 결과 DataFrame 생성 및 저장
df_backtest_results = pd.DataFrame(backtest_results_list)
print("\n===== 2024년 백테스트 결과 요약 =====")
print(df_backtest_results)
df_backtest_results.to_csv("2024_backtest_results.csv", index=False)

백테스팅 시작: HD
  완료: SMA | 수익률: -0.0172, MDD: 0.1074, Fitness: -0.0353
  완료: EMA | 수익률: -0.0809, MDD: 0.1879, Fitness: -0.1023
  완료: MACD | 수익률: 0.0628, MDD: 0.1074, Fitness: 0.0287
  완료: RSI | 수익률: 0.0000, MDD: 0.0000, Fitness: 0.0000
  완료: STOCH | 수익률: 0.0082, MDD: 0.1358, Fitness: -0.0206
  완료: ROC | 수익률: 0.0245, MDD: 0.1757, Fitness: -0.0155
  완료: BBANDS | 수익률: 0.0000, MDD: 0.0000, Fitness: 0.0000
  완료: ATR | 수익률: 0.0498, MDD: 0.0962, Fitness: 0.0206
  완료: CMF | 수익률: 0.0000, MDD: 0.0000, Fitness: 0.0000
  완료: OBV | 수익률: -0.0669, MDD: 0.1074, Fitness: -0.0750
백테스팅 시작: MCD
  완료: SMA | 수익률: -0.0379, MDD: 0.0889, Fitness: -0.0481
  완료: EMA | 수익률: -0.0083, MDD: 0.0889, Fitness: -0.0244
  완료: MACD | 수익률: -0.0049, MDD: 0.0889, Fitness: -0.0217
  완료: RSI | 수익률: 0.0000, MDD: 0.0000, Fitness: 0.0000
  완료: STOCH | 수익률: -0.2817, MDD: 0.2942, Fitness: -0.2842
  완료: ROC | 수익률: 0.0823, MDD: 0.0889, Fitness: 0.0480
  완료: BBANDS | 수익률: 0.0236, MDD: 0.1064, Fitness: -0.0024
  완료: ATR | 수익률: -0.0011, MD